In [ ]:
import pandas as pd
import re
import urllib.request
import json
import networkx as nx
import matplotlib.pyplot as plt
import requests
import os.path
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer 

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df_characters_unclean = pd.read_csv("/work/socialgraph-witcher-analysis/data/unclean_characters.csv")
df_characters_unclean.loc[125:134]

,pageid,title
125,3486,Prostitute
126,2573,Queen of the Night
127,4789,Radovid V
128,8038,Ramerot
129,2570,Ramsmeat
130,11206,Raven
131,2490,Rayla
132,2571,Raymond Maarloeve
133,2664,Ren Grouver
134,8874,Resolute girl


In [ ]:
len(df_characters_unclean)

836

In [ ]:
#REGEX

#Removes everything including and after the "Associated Quests" header until the next header.
# \=\=?\= Associated Quests \=\=?\=.*?[^\=]+

#Removes other headers.
# \=\=(.*?)\=\=

In [ ]:
#Saving data with weird é and ï in txt file names

baseurl = "https://witcher.fandom.com/api.php?"
action = "action=query"
content = "prop=extracts&exlimit=1&explaintext"
dataformat ="format=json"

for t in df_characters_unclean['title']:
    try:
        title_name = t.replace(" ", "_") #Ensure that whitespace -> underscores
        title_string = str(title_name.encode(encoding='UTF-8',errors='strict'))
        title_string = title_string[:-1].replace("b\'", "").replace("\\x", "%").replace("b\"", "")
        title = f"titles={title_string}"
        query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)

        response = urllib.request.urlopen(query)
        source = response.read()
        wikitext = source.decode('UTF-8')   
        wiki_json = json.loads(wikitext)
        page = list(wiki_json['query']['pages'].keys())[0]
        clean_text = wiki_json['query']['pages'][page]['extract']
        clean_text = clean_text.replace('\n', ' ').replace('\t', '')
        clean_text = re.sub(r'\=\=?\= Associated Quests \=\=?\=.*?[^\=]+', r'', clean_text)
        final_text = re.sub(r'\=\=(.*?)\=\=', r'', clean_text).replace('=', '')
        filename = title_name +'.txt'
        
        savepath = 'data_clean_texts'
        completeName = os.path.join(savepath, filename)
        
        with open(completeName, 'w') as f:
            f.write(final_text)
    except:
        print(t)

# Nedenstående tager ca. 25 min at køre!

In [ ]:
#Completely clean text versions

tk = WordPunctTokenizer()
lemmatizer = WordNetLemmatizer()

for character_name in df_characters_unclean["title"]:
    character_file_name = character_name.replace(' ','_') + ".txt"

    with open(f"data_clean_texts/{character_file_name}") as f:
        contents = str(f.readlines())
        tokens = tk.tokenize(contents)

        # remove stopwords
        tokens_without_sw = [word for word in tokens if not word in stopwords.words()]

        #remove punctuation
        tokens_no_punct= [word for word in tokens_without_sw if word.isalnum()]

        #lower everything
        tokens_lower = [word.lower() for word in tokens_no_punct]

        #Remove names and name parts (ie. if only first name is written in the text)
        #Regex splits at both ' and white spaces in the name.
        for name in df_characters_unclean['title'].values:
            name = name.lower()
            name_split = re.split('[\'\s]',name)
            for name_part in name_split:
                if name_part in tokens_lower:
                    try:
                        while True:
                            tokens_lower.remove(name_part)
                    except ValueError:
                        pass
        
        #Lemmatize words
        tokens_lemma = [lemmatizer.lemmatize(word) for word in tokens_lower]

        #Convert to text
        cleaned_text = nltk.Text(tokens_lemma)
        
        #Save to file
        #new_file = open(f"data_all_clean/{character_name}.txt", "w")
        #for i in range(len(cleaned_text)):
        #    new_file.write(f"{cleaned_text[i]} ")
        #new_file.close()

In [ ]:
#Test

tk = WordPunctTokenizer()
lemmatizer = WordNetLemmatizer()

character_file_name = "Éibhear_Hattori.txt"

with open(f"data_clean_texts/{character_file_name}") as f:
    contents = str(f.readlines())
    tokens = tk.tokenize(contents)

    # remove stopwords
    tokens_without_sw = [word for word in tokens if not word in stopwords.words()]

    #remove punctuation
    tokens_no_punct= [word for word in tokens_without_sw if word.isalnum()]

    #lower everything
    tokens_lower = [word.lower() for word in tokens_no_punct]

    #Remove names and name parts (ie. if only first name is written in the text)
    #Regex splits at both ' and white spaces in the name.
    for name in df_characters_unclean['title'].values:
        name = name.lower()
        name_split = re.split('[\'\s]',name)
        for name_part in name_split:
            if name_part in tokens_lower:
                try:
                    while True:
                        tokens_lower.remove(name_part)
                except ValueError:
                    pass
    
    #Lemmatize words
    tokens_lemma = [lemmatizer.lemmatize(word) for word in tokens_lower]

    #Convert to text
    cleaned_text = nltk.Text(tokens_lemma)
    print(cleaned_text)

<Text: living house glory lane district novigrad around 1272...>


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=061c6de9-714b-4aa5-8dca-7fde3082e296' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>